In [6]:

from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 8, Finished, Available, Finished)

In [1]:
df = spark.read.option("multiline", "true").json("Files/2025-03-19_project_fabrics_earthquake_data.json")
# df now is a Spark DataFrame containing JSON data from "Files/2025-03-19_project_fabrics_earthquake_data.json".
display(df)

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dd62d6c5-7569-4acd-a369-f74a30a2b92d)

here data is hiding inside the geometry column. Each coordinates has 3 different values.

In [2]:
display(df.select("geometry.coordinates"))

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9b07d70c-e3a2-47b4-8efe-3cab68145bf3)

Here I display the first coordinate in geometry column


In [7]:
display(df.select(col("geometry.coordinates").getItem(0)))

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a54c8d33-634e-4d07-8169-588c7ea99657)

Reshape earthquake data by extracting and renaming key attributes for further analysis.

In [8]:
df = \
df.\
    select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
        )
     

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 10, Finished, Available, Finished)

 Now I convert 'time' and 'updated' columns from milliseconds to timestamp format for clearer datetime representation.

In [9]:
df = df.\
    withColumn('time', col('time')/1000).\
    withColumn('updated', col('updated')/1000).\
    withColumn('time', col('time').cast(TimestampType())).\
    withColumn('updated', col('updated').cast(TimestampType()))
     

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 11, Finished, Available, Finished)

Now I created the table inside the lakehouse

In [11]:
df.write.mode('append').saveAsTable('Fabrics_project_earthquake_silver_layer')

StatementMeta(, b610ba94-c7cb-42ab-b501-7ab1d73d16a2, 13, Finished, Available, Finished)